In [1]:
import boto3
import pandas as pd
import numpy as np
import warnings
from tqdm import tqdm
warnings.filterwarnings("ignore")

In [ ]:
client = boto3.client('s3')

In [ ]:
paginator = client.get_paginator('list_objects_v2')

# List objects under the specified prefix
page_iterator = paginator.paginate(Bucket=bucket, Prefix='AthenaInsights/latest_data/', Delimiter='/')

for page in page_iterator:
    if 'CommonPrefixes' in page:
        for folder in page['CommonPrefixes']:
            print(folder['Prefix'])

AthenaInsights/latest_data/csv/
AthenaInsights/latest_data/data_prep/
AthenaInsights/latest_data/dependent_variable/
AthenaInsights/latest_data/feature_prep/
AthenaInsights/latest_data/parquet/
AthenaInsights/latest_data/reduced_autocorelation/


### reading data

In [ ]:
s3_prefix = 's3://'
bucket = 'sisyphus-general-bucket'
primary_folder = 'AthenaInsights'

In [ ]:
response = client.list_objects_v2(
    Bucket=bucket,
    Prefix=f'{primary_folder}/latest_data/feature_prep/')

In [ ]:
paths = []
for content in response.get('Contents', []):
    # print(f"{s3_prefix}{bucket}/{content['Key']}")
    k = content['Key']
    if 'base' in k and '1min' in k: #('base' in k or 'diff' in k) and ('rsi' not in k and 'macd' not in k):
        paths.append(f"{s3_prefix}{bucket}/{content['Key']}")
paths

['s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_1min_base.parquet',
 's3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_1min_base_macd.parquet',
 's3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_1min_base_other_features.parquet',
 's3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_1min_base_rsi.parquet']

In [30]:
paths = []
for content in response.get('Contents', []):
    # print(f"{s3_prefix}{bucket}/{content['Key']}")
    k = content['Key']
    if 'base' in k and '2min' in k: #('base' in k or 'diff' in k) and ('rsi' not in k and 'macd' not in k):
        paths.append(f"{s3_prefix}{bucket}/{content['Key']}")
paths

['s3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_2min_base.parquet',
 's3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_2min_base_macd.parquet',
 's3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_2min_base_rsi.parquet']

In [2]:
from pyarrow.parquet import ParquetFile
import pyarrow as pa 

path = 's3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_1min_base_other_features.parquet'

pf = ParquetFile(path) 
first_ten_rows = next(pf.iter_batches(batch_size = 10)) 
df = pa.Table.from_batches([first_ten_rows]).to_pandas() 

df

,us_eastern_timestamp,open,high,low,close,volume,open_ema_2m,open_sma_2m,high_ema_2m,high_sma_2m,...,cumulative_local_max_15,cumulative_local_min_15,time_since_prev_max_15,time_since_prev_min_15,local_max_30,local_min_30,cumulative_local_max_30,cumulative_local_min_30,time_since_prev_max_30,time_since_prev_min_30
0,2024-01-02 04:00:00,476.25,476.36,476.00,476.31,20460.0,476.250000,NaN,476.360000,NaN,...,0,0,0,0,False,False,0,0,0,0
1,2024-01-02 04:01:00,476.34,476.34,476.29,476.29,6369.0,476.310000,476.295,476.346667,476.350,...,0,0,1,1,False,False,0,0,1,1
2,2024-01-02 04:02:00,476.29,476.29,476.28,476.28,6152.0,476.296667,476.315,476.308889,476.315,...,0,0,2,2,False,False,0,0,2,2
3,2024-01-02 04:03:00,476.27,476.27,476.27,476.27,369.0,476.278889,476.280,476.282963,476.280,...,0,0,3,3,False,False,0,0,3,3
4,2024-01-02 04:04:00,476.27,476.27,476.27,476.27,369.0,476.272963,476.270,476.274321,476.270,...,0,0,4,4,False,False,0,0,4,4
5,2024-01-02 04:05:00,476.20,476.20,476.19,476.19,547.0,476.224321,476.235,476.224774,476.235,...,0,0,5,5,False,False,0,0,5,5
6,2024-01-02 04:06:00,476.18,476.18,476.18,476.18,230.0,476.194774,476.190,476.194925,476.190,...,0,0,6,6,False,False,0,0,6,6
7,2024-01-02 04:07:00,476.21,476.21,476.14,476.14,5624.0,476.204925,476.195,476.204975,476.195,...,0,0,7,7,False,False,0,0,7,7
8,2024-01-02 04:08:00,476.19,476.19,476.19,476.19,107.0,476.194975,476.200,476.194992,476.200,...,0,0,8,8,False,False,0,0,8,8
9,2024-01-02 04:09:00,476.19,476.19,476.19,476.19,107.0,476.191658,476.190,476.191664,476.190,...,0,0,9,9,False,False,0,0,9,9


In [34]:
# HTTPException: HTTP Error: HTTP GET error on 'https://sisyphus-general-bucket.s3.amazonaws.com/AthenaInsights/latest_data/feature_prep/stock_bars_1min_base_other_features.parquet' (HTTP 403)

# import duckdb
# nrows = 10
# df = duckdb.query(f'SELECT * FROM "{path}" LIMIT 10;').df()

HTTPException: HTTP Error: HTTP GET error on 'https://sisyphus-general-bucket.s3.amazonaws.com/AthenaInsights/latest_data/feature_prep/stock_bars_1min_base_other_features.parquet' (HTTP 403)

In [1]:
import pyarrow.dataset as ds
path = 's3://sisyphus-general-bucket/AthenaInsights/latest_data/feature_prep/stock_bars_1min_base_other_features.parquet'
n = 10
df = ds.dataset(path).scanner().head(n).to_pandas()
df

,us_eastern_timestamp,open,high,low,close,volume,open_ema_2m,open_sma_2m,high_ema_2m,high_sma_2m,...,cumulative_local_max_15,cumulative_local_min_15,time_since_prev_max_15,time_since_prev_min_15,local_max_30,local_min_30,cumulative_local_max_30,cumulative_local_min_30,time_since_prev_max_30,time_since_prev_min_30
0,2024-01-02 04:00:00,476.25,476.36,476.00,476.31,20460.0,476.250000,NaN,476.360000,NaN,...,0,0,0,0,False,False,0,0,0,0
1,2024-01-02 04:01:00,476.34,476.34,476.29,476.29,6369.0,476.310000,476.295,476.346667,476.350,...,0,0,1,1,False,False,0,0,1,1
2,2024-01-02 04:02:00,476.29,476.29,476.28,476.28,6152.0,476.296667,476.315,476.308889,476.315,...,0,0,2,2,False,False,0,0,2,2
3,2024-01-02 04:03:00,476.27,476.27,476.27,476.27,369.0,476.278889,476.280,476.282963,476.280,...,0,0,3,3,False,False,0,0,3,3
4,2024-01-02 04:04:00,476.27,476.27,476.27,476.27,369.0,476.272963,476.270,476.274321,476.270,...,0,0,4,4,False,False,0,0,4,4
5,2024-01-02 04:05:00,476.20,476.20,476.19,476.19,547.0,476.224321,476.235,476.224774,476.235,...,0,0,5,5,False,False,0,0,5,5
6,2024-01-02 04:06:00,476.18,476.18,476.18,476.18,230.0,476.194774,476.190,476.194925,476.190,...,0,0,6,6,False,False,0,0,6,6
7,2024-01-02 04:07:00,476.21,476.21,476.14,476.14,5624.0,476.204925,476.195,476.204975,476.195,...,0,0,7,7,False,False,0,0,7,7
8,2024-01-02 04:08:00,476.19,476.19,476.19,476.19,107.0,476.194975,476.200,476.194992,476.200,...,0,0,8,8,False,False,0,0,8,8
9,2024-01-02 04:09:00,476.19,476.19,476.19,476.19,107.0,476.191658,476.190,476.191664,476.190,...,0,0,9,9,False,False,0,0,9,9


In [41]:
# ClientError: An error occurred (MethodNotAllowed) when calling the SelectObjectContent operation: The specified method is not allowed against this resource.

# import awswrangler as wr

# df = wr.s3.select_query(
#         sql="SELECT * FROM s3object s limit 10",
#         path=path,
#         input_serialization="Parquet",
#         input_serialization_params={},
#         use_threads=True,
# )